![LogoUC3M](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a6/Acr%C3%B3nimo_y_nombre_de_la_UC3M.svg/320px-Acr%C3%B3nimo_y_nombre_de_la_UC3M.svg.png) 

*Alonso Rios Guerra - 100495821 | Guillermo Sancho González - 100495991*


# *__Aprendizaje automático P1: Predicción del abandono de empleados__*

## *__1. Introducción__*

En esta práctica tenemos como objetivo desarrollar diferentes métodos de aprendizaje automático para predecir el abandono de los trabajadores de una empresa.

Primero de todo empezaremos leyendo los datos que nos proporciona la empresa. En nuestro caso, usaremos el dataset Nº10.

In [1]:
import numpy as np
import pandas as pd

data_path = 'attrition_availabledata_10.csv.gz'

data = pd.read_csv(data_path, compression='gzip', sep = ',')

## *__2. EDA Simplificado__*



Un EDA es una análisis exploratorio de datos, para organizar los datos, entender su contenido, entender cual son las variables más relevantes y cómo se relacionan unas con otras, determinar qué hacer con los datos faltantes y con los datos atípicos, y finalmente extraer conclusiones acerca de todo este análisis.

Para hacer un eda debemos responder a distintas preguntas:

-   ¿Cuántas instancias y atributos hay?

-   ¿Qué tipo de atributos hay (numéricos o categóricos)? Esto se hace para verificar si hay características categóricas que deben ser codificadas (como variables dummy o one-hot encoding). Comprobar si hay variables categóricas con alta cardinalidad.

-   ¿Qué atributos tienen valores faltantes y cuántos?

-   ¿Existen columnas constantes o ID?

-   ¿Es un problema de clasificación o regresión (variable de respuesta) y? En caso de clasificación, ¿las clases están desbalanceadas?

A continuación le damos respuesta:

-   ¿Cuántas instancias y atributos hay?

In [46]:
print('La forma de la tabla es:', data.shape)

La forma de la tabla es: (2940, 27)


El dataset contiene 2940 instancias, 30 atributos y 1 etiqueta (Attrition).

-   ¿Qué tipo de atributos hay (numéricos o categóricos)? Esto se hace para verificar si hay características categóricas que deben ser codificadas (como variables dummy o one-hot encoding). Comprobar si hay variables categóricas con alta cardinalidad.

In [48]:
print('Los tipos de atributos son:')
print('==================================')
print(data.dtypes)

Los tipos de atributos son:
hrs                        float64
absences                     int64
JobInvolvement               int64
PerformanceRating            int64
EnvironmentSatisfaction    float64
JobSatisfaction            float64
WorkLifeBalance            float64
Age                          int64
BusinessTravel              object
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
Gender                      object
JobLevel                     int64
JobRole                     object
MaritalStatus               object
MonthlyIncome                int64
NumCompaniesWorked         float64
PercentSalaryHike            int64
StockOptionLevel             int64
TotalWorkingYears          float64
TrainingTimesLastYear        int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
YearsWithCurrManager         int64
Attrition                   object
dtype: object


Existen dos tipos de atributos en nuestro dataset: numéricos y categóricos. Dentro de los numéricos encontramos de tipo entero (absences, Age, JobLevel, ...) y de tipo float (hrs, TotalWorkingYears, JobSatisfaction). En cuanto a lo atributos categóricos encontramos algunos como Department, JobRole, MaritalStatus, ... Para entrenar a nuestro modelo nos interesa codificar las variables categóricas y por ello es importante ver como de viable es según su cardinalidad.

In [4]:
columnas_cat = data.select_dtypes(include=['object']).columns # Selecciona las columnas categóricas

print('Cardinalidad de los atributos categóricos:')
print('================================')
for col in columnas_cat: # Imprime la cardinalidad de cada atributo categórico
    print(f"{col}: {data[col].nunique()} categorías únicas")

Cardinalidad de los atributos categóricos:
BusinessTravel: 3 categorías únicas
Department: 3 categorías únicas
EducationField: 6 categorías únicas
Gender: 2 categorías únicas
JobRole: 9 categorías únicas
MaritalStatus: 3 categorías únicas
Over18: 1 categorías únicas
Attrition: 2 categorías únicas


Al observar la ejecución del código anterior vemos que la cardinalidad de nuestros atributos categóricos es baja, en un rango de [2-9], y por ello no implicará ningún problema a la hora realizar una codificación dummy o One-Hot Encoding.

-   ¿Qué atributos tienen valores faltantes y cuántos?

In [49]:
print('Cuántos valores faltan por atributo:')
print('====================================')
sin_valor = data.isnull().sum()  # Cuenta valores nulos por columna
sin_valor = sin_valor[sin_valor > 0]  # Filtra solo los que tienen valores nulos

print(sin_valor)

Cuántos valores faltan por atributo:
EnvironmentSatisfaction    15
JobSatisfaction            12
WorkLifeBalance            29
NumCompaniesWorked         17
TotalWorkingYears           5
dtype: int64


Tras ejecutar el código anterior, obtenemos que existen 5 atributos con valores faltantes. Estos atributos son: 
EnvironmentSatisfaction con 15 faltantes, 
JobSatisfaction con 12 faltantes,
WorkLifeBalance con 29 faltantes,
NumCompaniesWorked con 17 faltantes y
TotalWorkingYears con 5 faltantes.

- ¿Existen columnas constantes o ID?

In [6]:
# 1. Comprobar columnas constantes
constantes = [col for col in data.columns if data[col].nunique() == 1]
print("Columnas constantes:", constantes)

# 2. Comprobar columnas ID
columnas_id = [col for col in data.columns if data[col].nunique() == len(data)]
print("Columnas ID:", columnas_id)


Columnas constantes: ['EmployeeCount', 'Over18', 'StandardHours']
Columnas ID: ['EmployeeID']


Observamos que existen 3 columnas constantes (EmployeeCount, Over18, StandardHours) y una columna ID (EmployeeID)

- ¿Es un problema de clasificación o regresión (variable de respuesta) y? En caso de clasificación, ¿las clases están desbalanceadas?
 

En este caso es fácil ver que es un problema de __clasificación__ porque la etiqueta (Attrition) en los datos train solo pueden tener valores 'Yes' o 'No', por lo que es una clase binaria.

In [7]:
print('Comprobar si la clase está desbalanceada:')
print('======================================')
print(data['Attrition'].value_counts())
print()
print(data['Attrition'].value_counts() / data['Attrition'].count())

Comprobar si la clase está desbalanceada:
Attrition
No     2466
Yes     474
Name: count, dtype: int64

Attrition
No     0.838776
Yes    0.161224
Name: count, dtype: float64


Se puede ver que la clase esta bastante desbalanceada: 83.88% No, 16.12% Yes

## *__3. ¿Cómo se va a realizar la evaluación?__*

Para realizar la evaluación de nuestro modelo vamos a seguir una serie de pasos. La evaluación estará divida en inner, donde se elegirá el mejor scaler, imputer y ajuste de hiperparámetros, y outer, donde se estimará el rendimiento a futuro del modelo. Es importante dividir nuestro dataset en datos de entrenamiento (usados en inner) y de validación (usados en outer). En nuestro caso seguiremos el Holdout ((2/3) Train y (1/3) Test).

- Inner Evaluation

    - Primero

- Outer Evaluation

## *__4. Metodos básicos: KNN y Tree__*

Inicialmente vamos a eliminar las columnas constantes o ids que consideramos que no aportan información a la investigación

In [8]:
data = data.drop(columns=constantes + columnas_id)

Se dividen los datos en x e y, donde x son los inputs, e y es la etiqueta.

Despues se vuelven a dividir en train y en test. Train se usará para la evaluación interna, y test se usará para la evaluación final.


In [88]:
from sklearn.model_selection import train_test_split


X = data.drop(columns=['Attrition'])
y = data['Attrition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

Se itera creando pipelines con los distintos imputers y scalers para observar cual es el que tiene más precisión en el modelo

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, cross_val_score
from sklearn import metrics

#Separar variables categoricas y numéricas
columnas_num = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
columnas_cat = X.select_dtypes(include=['object']).columns.tolist()


#Distintos métodos de escalado e imputación
scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
imputers = ['mean', 'median']

#Se realiza una crossvalidation con 3 folds
cv = KFold(n_splits=3, shuffle=True, random_state=42)

accuracy = -np.inf
best_scaler, best_imputer = None, None

#Definir los pasos en la Pipeline
for i in range(len(imputers)):
    for j in range(len(scalers)):
        knn = KNeighborsClassifier()
        scaler = scalers[j]
        imputer = SimpleImputer(strategy=imputers[i])
        encoder = OneHotEncoder()

        classif_numericos = Pipeline([
            ("imputation", imputer),
            ("standardization", scaler)
        ])

        classif_categoricos = Pipeline([
            ("encoder", encoder),
            ("imputation", imputer)
        ])

        preprocessor = ColumnTransformer(
            transformers=[
                ("num", classif_numericos, columnas_num),
                ("cat", classif_categoricos, columnas_cat)
            ]
        )

        clf = Pipeline([("preprocessor", preprocessor), ("knn", knn)])

        scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=cv)
        print(scores.mean())
        if scores.mean() > accuracy:
            accuracy = scores.mean()
            best_scaler, best_imputer = scalers[j], imputers[i]
        """print(f"La precisión media de la validación cruzada es: {scores.mean():.4f} ± {scores.std():.4f}")"""

print(f"Best accuracy: {accuracy:.4f}. With scaler: {best_scaler}, imputer: {best_imputer}")

0.8505102040816326
0.8367346938775511
0.8535714285714286
0.8505102040816326
0.8362244897959183
0.8540816326530614
Best accuracy: 0.8541. With scaler: RobustScaler(), imputer: median


El mejor scaler es el StandardScaler(), y el mejor imputer el SimpleImputer(strategy='mean'), es decir, utilizando la media para imputar los datos nulos.

Una vez sabemos cual es el mejor scaler y el mejor imputer, hacemos el mismo proceso hecho antes, pero para elegir los mejores hiperparámetros del KNN.

In [ ]:
classif_numericos = Pipeline([
    ("imputation", SimpleImputer(strategy=best_imputer)),
    ("standardization", best_scaler)
])

classif_categoricos = Pipeline([
    ("encoder", OneHotEncoder()),
    ("imputation", SimpleImputer(strategy=best_imputer))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", classif_numericos, columnas_num),
        ("cat", classif_categoricos, columnas_cat)
    ]
)
accuracy = -np.inf
best_n, best_p = 0, 0

for n in [1, 2, 3, 4, 8, 16, 32]: # Elegir el numero de neighbors
    for p in [1, 2]: # Usar la distancia de Manhattan (1) o Euclídea

        clf = Pipeline([("preprocessor", preprocessor), ("classifier", KNeighborsClassifier(p=p, n_neighbors=n))])
        
        scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=cv)
        if scores.mean() > accuracy:
            accuracy = scores.mean()
            best_n, best_p = n, p

print(f"Best accuracy: {accuracy:.4f}. With n: {best_n}, p: {best_p}")

Best accuracy: 0.8969. With n: 1, p: 2


Los hiperparámetros optimizados obtenidos son que el valor de K (n-neighbors) sea 1 y se utilize la distancia euclídea (p=2), con una precisión de 0,8969.